# MVP-L: KITTI 라이다 데이터 로딩

이 노트북은 KITTI 데이터셋의 라이다 포인트 클라우드와 칼리브레이션 정보를 로딩하고 시각화합니다.

## 목표
- KITTI 포인트 클라우드 로딩 (.bin 파일)
- 칼리브레이션 정보 로딩
- 포인트 클라우드 시각화 (Open3D)

In [ ]:
# Cell 1: 라이브러리 설치 및 임포트
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt
from pathlib import Path

print("Open3D 버전:", o3d.__version__)
print("라이브러리 로드 완료")

In [ ]:
# Cell 2: 경로 설정 및 데이터셋 구조 확인
project_root = Path.cwd().parent.parent
dataset_root = project_root / "dataset" / "training"

# KITTI 라이다 데이터는 일반적으로 velodyne 폴더에 있음
velodyne_dir = dataset_root / "velodyne"
calib_dir = dataset_root / "calib"

print(f"프로젝트 루트: {project_root}")
print(f"Velodyne 디렉토리: {velodyne_dir} (존재: {velodyne_dir.exists()})")
print(f"Calib 디렉토리: {calib_dir} (존재: {calib_dir.exists()})")

In [ ]:
# Cell 3: 포인트 클라우드 로딩 함수
def load_pointcloud(bin_path):
    """
    KITTI .bin 파일에서 포인트 클라우드 로딩
    
    KITTI 포맷: [x, y, z, intensity] float32, 바이너리
    """
    pointcloud = np.fromfile(str(bin_path), dtype=np.float32).reshape(-1, 4)
    return pointcloud[:, :3], pointcloud[:, 3]  # points (x,y,z), intensity

def load_pointcloud_o3d(bin_path):
    """
    Open3D 포인트 클라우드 객체로 로딩
    """
    points, intensity = load_pointcloud(bin_path)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # intensity를 색상으로 매핑
    colors = np.zeros_like(points)
    intensity_norm = (intensity - intensity.min()) / (intensity.max() - intensity.min() + 1e-6)
    colors[:, 0] = intensity_norm  # 빨강 채널에 intensity
    colors[:, 1] = intensity_norm  # 초록 채널
    colors[:, 2] = intensity_norm  # 파랑 채널
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    return pcd

print("포인트 클라우드 로딩 함수 정의 완료")

In [ ]:
# Cell 4: 칼리브레이션 정보 로딩
def load_calibration(calib_path):
    """
    KITTI 칼리브레이션 파일 로딩
    """
    calib = {}
    if not calib_path.exists():
        return calib
    
    with open(calib_path, 'r') as f:
        for line in f:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                values = [float(x) for x in value.strip().split()]
                if key.strip() == 'Tr_velo_to_cam':
                    # 3x4 변환 행렬
                    calib[key.strip()] = np.array(values).reshape(3, 4)
                elif key.strip() == 'Tr_velo_to_cam':
                    calib[key.strip()] = np.array(values).reshape(3, 4)
                elif key.strip().startswith('P'):
                    # 투영 행렬
                    calib[key.strip()] = np.array(values).reshape(3, 4)
                elif key.strip().startswith('R'):
                    # 회전 행렬
                    if len(values) == 9:
                        calib[key.strip()] = np.array(values).reshape(3, 3)
    
    return calib

# 테스트
if calib_dir.exists():
    sample_calib = list(calib_dir.glob("*.txt"))[0] if list(calib_dir.glob("*.txt")) else None
    if sample_calib:
        calib = load_calibration(sample_calib)
        print(f"칼리브레이션 키: {list(calib.keys())}")
        if 'Tr_velo_to_cam' in calib:
            print(f"Tr_velo_to_cam:\\n{calib['Tr_velo_to_cam']}")
else:
    print("칼리브레이션 파일을 찾을 수 없습니다.")

In [ ]:
# Cell 5: 포인트 클라우드 시각화
def visualize_pointcloud(pcd, title="Point Cloud"):
    """
    Open3D를 사용한 포인트 클라우드 시각화
    """
    o3d.visualization.draw_geometries([pcd], window_name=title, width=1024, height=768)

def visualize_pointcloud_matplotlib(points, intensity=None, title="Point Cloud"):
    """
    Matplotlib을 사용한 포인트 클라우드 시각화 (BEV - Top View)
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Top View (X-Y 평면)
    ax = axes[0]
    if intensity is not None:
        scatter = ax.scatter(points[:, 0], points[:, 1], c=intensity, 
                           cmap='jet', s=0.5, alpha=0.5)
        plt.colorbar(scatter, ax=ax, label='Intensity')
    else:
        ax.scatter(points[:, 0], points[:, 1], s=0.5, alpha=0.5)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(f'{title} - Top View (X-Y)')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    # Side View (X-Z 평면)
    ax = axes[1]
    if intensity is not None:
        scatter = ax.scatter(points[:, 0], points[:, 2], c=intensity, 
                           cmap='jet', s=0.5, alpha=0.5)
        plt.colorbar(scatter, ax=ax, label='Intensity')
    else:
        ax.scatter(points[:, 0], points[:, 2], s=0.5, alpha=0.5)
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Z (m)')
    ax.set_title(f'{title} - Side View (X-Z)')
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    return fig

print("시각화 함수 정의 완료")

In [ ]:
# Cell 6: 샘플 포인트 클라우드 로드 및 시각화
if velodyne_dir.exists():
    bin_files = sorted(velodyne_dir.glob("*.bin"))
    if bin_files:
        sample_bin = bin_files[0]
        print(f"샘플 포인트 클라우드: {sample_bin.name}")
        
        # 포인트 클라우드 로드
        points, intensity = load_pointcloud(sample_bin)
        print(f"포인트 개수: {len(points):,}")
        print(f"포인트 범위:")
        print(f"  X: [{points[:, 0].min():.2f}, {points[:, 0].max():.2f}] m")
        print(f"  Y: [{points[:, 1].min():.2f}, {points[:, 1].max():.2f}] m")
        print(f"  Z: [{points[:, 2].min():.2f}, {points[:, 2].max():.2f}] m")
        
        # Matplotlib 시각화
        fig = visualize_pointcloud_matplotlib(points, intensity, 
                                            title=f"Sample: {sample_bin.name}")
        
        # Open3D 시각화 (선택사항)
        # pcd = load_pointcloud_o3d(sample_bin)
        # visualize_pointcloud(pcd, title=f"Sample: {sample_bin.name}")
    else:
        print("포인트 클라우드 파일을 찾을 수 없습니다.")
else:
    print("Velodyne 디렉토리를 찾을 수 없습니다.")